*PART 1: Intro and Data Cleaning*

***

# 1. Lending Club

By Rokas Burneika for Turing College

**Context**

I was hired by LendingClub for a new project. 

They wanted to fully automate their decisions.

As it appears, there was a try it some time ago and the datasets were saved, thus meaning we have a great base for a model.

**3-Step Process**

To tackle this problem, we have settled on 3-step proccess:

1. Creating a Machine Learning model to classify loans into accepted/rejected;
2. Second model predicting a grade of loan;
3. Third model predicting subgrade and interest rate.

*Dataset is downloaded from [here](https://storage.googleapis.com/335-lending-club/lending-club.zip)*

***

# 2. Data Cleaning and Preparation 

We have two datasets at our disposal: accepted and rejecte loans.

To prepare them for Machine Learning models we will have to merge them into one.

## 2.1. Data and Module Import

In [1]:
import pandas as pd
import numpy as np
import warnings
from utilities import *

warnings.filterwarnings('ignore')

file_path1 = './lending-club/accepted_2007_to_2018q4.csv/accepted_2007_to_2018q4.csv'
file_path2 = './lending-club/rejected_2007_to_2018q4.csv/rejected_2007_to_2018q4.csv'

df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)

In [2]:
df1.shape

(2260701, 151)

Accepted loans dataset has over 2 million rows and 151 features.

In [3]:
df2.shape

(27648741, 9)

As for rejected loans dataset, it has over 27 million records and 9 columns.

Meaning we will have to pick out which features from accepted data align with ones from rejected.

In [4]:
df2.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0


Columns of rejected loans data:

1. `Amount Requested`: how much money was requested for the loan;
2. `Application Date`: when was the request submited;
3. `Loan Title`: request title;
4. `Risk_Score`: a FICO score (it will be automatically calculated by another deployed model and will be fed into this one);
5. `Debt-To-Income Ratio` (DTI for short): percentage of how big the loan's monthly instatement is compared to monthly income. 
6. `Zip Code`: zip code;
7. `State`: USA state;
8. `Employment Length`: for how long has the borrower been working for his/hers current employer;
9. `Policy Code`: I found no information what it actually is, but for now let's say it's useful.

Also, DataFrame of rejected loans also has these values:

1. `loan_amnt`: for `Amount Requested`;
2. `title`: for `Loan Title`;
3. `fico_range_high`: for `Risk_Score`;
4. `dti`: for `Debt-To-Income Ratio`;
5. `zip_code`: for `Zip Code`;
6. `addr_state`: for `State`;
7. `emp_length`: for `Employment Length`;
8. `policy_code`: for `Policy Code`.

Here I am already not including `Application Sate` as it will not be useful for our model.

So here I am dropping it from accepted loans.

In [5]:
df2.drop(labels=['Application Date'], axis=1, inplace=True)

### Changing Column Names

Saving accepted loans dataset with only values that match rejected dataset.

In [6]:
df1 = df1[['loan_amnt', 'title', 'fico_range_high', 'dti', 'zip_code', 'addr_state', 'emp_length', 'policy_code']]
df1.head()

,loan_amnt,title,fico_range_high,dti,zip_code,addr_state,emp_length,policy_code
0,3600.0,Debt consolidation,679.0,5.91,190xx,PA,10+ years,1.0
1,24700.0,Business,719.0,16.06,577xx,SD,10+ years,1.0
2,20000.0,NaN,699.0,10.78,605xx,IL,10+ years,1.0
3,35000.0,Debt consolidation,789.0,17.06,076xx,NJ,10+ years,1.0
4,10400.0,Major purchase,699.0,25.37,174xx,PA,3 years,1.0


And then changing names on features in rejected loans dataset to match the ones on accepted data.

In [7]:
df2.columns = ['loan_amnt', 'title', 'fico_range_high', 'dti', 'zip_code', 'addr_state', 'emp_length', 'policy_code']
df2.head()

,loan_amnt,title,fico_range_high,dti,zip_code,addr_state,emp_length,policy_code
0,1000.0,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0


### DTI

We can see that for rejected loans, DTI values are in percentages rather than floats.

This function removes the percentage sign at the end.

In [8]:
def percentage_to_float(x: str):
    return x[:-1]

In [9]:
df2['dti'] = df2['dti'].apply(percentage_to_float).astype('float64')
df2['dti']

0           10.00
1           10.00
2           10.00
3           38.64
4            9.43
            ...  
27648736    41.26
27648737     1.48
27648738    10.26
27648739    17.71
27648740    10.58
Name: dti, Length: 27648741, dtype: float64

### Policy Code

Let's check values for Policy Code.

In [10]:
df1['policy_code'].value_counts()

1.0    2260668
Name: policy_code, dtype: int64

In [11]:
df2['policy_code'].value_counts()

0.0    27559694
2.0       88129
Name: policy_code, dtype: int64

Well, since all accepted loans have policy code 1 and all rejected ones wither 0 or 2, it means that is our target feature is completely correlated with policy code.

Therefore, we must remove it due to data leakage.

In [12]:
df1.drop(columns='policy_code', inplace=True)
df2.drop(columns='policy_code', inplace=True)

## Concatenating DataFrames

Finally joining the two datasets into one.

In [13]:
df1['target'] = 1
df2['target'] = 0

ml_df = pd.concat([df1, df2])
ml_df = ml_df.rename(columns={'fico_range_high': 'risk_score'})
ml_df.reset_index(inplace=True, drop=True)

ml_df[:5]

,loan_amnt,title,risk_score,dti,zip_code,addr_state,emp_length,target
0,3600.0,Debt consolidation,679.0,5.91,190xx,PA,10+ years,1
1,24700.0,Business,719.0,16.06,577xx,SD,10+ years,1
2,20000.0,NaN,699.0,10.78,605xx,IL,10+ years,1
3,35000.0,Debt consolidation,789.0,17.06,076xx,NJ,10+ years,1
4,10400.0,Major purchase,699.0,25.37,174xx,PA,3 years,1


## Cleaning Full Dataset

### Null Values

In [14]:
ml_df.dropna(inplace=True)
ml_df.shape

(11086103, 8)

After removing null values, we have cut down the length of data from around 30 million to only a little over 11 million.

### Zip Codes and States

Regarding Zip Codes and States, I have found that these features do not correlate together.

Based on information on [Wikipedia about Zip Code prefixes](https://en.wikipedia.org/wiki/List_of_ZIP_Code_prefixes), it seems there are some false codes on `zip_code` colummn.

For example, zip codes taht start with 213, 343, 345, 429, 536 are not in use.

However:

In [15]:
ml_df[ml_df['zip_code'].isin(['213xx', '343xx', '345xx', '429xx', '536xx'])]['zip_code'].value_counts()

345xx    39
343xx    28
213xx    21
536xx    11
429xx     4
Name: zip_code, dtype: int64

We can see that they are used in this dataset. 

This could provide useful to detect bad loans.

According to before mentioned Wikipedia page, Ohio should have only 29 zip code prefixes (430-458).

In [16]:
ml_df.head()

,loan_amnt,title,risk_score,dti,zip_code,addr_state,emp_length,target
0,3600.0,Debt consolidation,679.0,5.91,190xx,PA,10+ years,1
1,24700.0,Business,719.0,16.06,577xx,SD,10+ years,1
3,35000.0,Debt consolidation,789.0,17.06,076xx,NJ,10+ years,1
4,10400.0,Major purchase,699.0,25.37,174xx,PA,3 years,1
5,11950.0,Debt consolidation,694.0,10.20,300xx,GA,4 years,1


In [17]:
ml_df[ml_df['addr_state'] == 'OH']['zip_code'].nunique()

359

We can see that in this dataset Ohio has 359 prefixes assigned to it.

In [18]:
ml_df[(ml_df['addr_state'] == 'OH') & (ml_df['target'] == 1)]['zip_code'].nunique()

76

And there are still 76 different prefixes if we are looking only at accepted loans.

Based on this, either `zip_code` or `addr_state` (or both) are providing false information.

Due to `zip_code` containing unused zip codes I will be removing the column entirely and only keeping `addr_state`.

In [19]:
ml_df.drop(columns='zip_code', inplace=True)
ml_df.head()

,loan_amnt,title,risk_score,dti,addr_state,emp_length,target
0,3600.0,Debt consolidation,679.0,5.91,PA,10+ years,1
1,24700.0,Business,719.0,16.06,SD,10+ years,1
3,35000.0,Debt consolidation,789.0,17.06,NJ,10+ years,1
4,10400.0,Major purchase,699.0,25.37,PA,3 years,1
5,11950.0,Debt consolidation,694.0,10.20,GA,4 years,1


## Removing Outliers

In [20]:
ml_df = drop_numerical_outliers(ml_df)
ml_df.shape

(10985383, 7)

After removing outliers, we still have around 11 million records.

## Saving Data

Finally, to save our data for later use.

In [21]:
ml_df.reset_index(drop=True, inplace=True)
ml_df.to_csv('prepared_data.csv')

***

# End of First Part

**Summary**

What we have done:

1. Created a 3-step process to tackle this problem;
2. Cleaned the data of `Null` values;
3. Prepared some features for further classification.

What's next?

1. Basic EDA;
2. Model training!